# **TIỀN XỬ LÝ** 

## Các bước thực hiện:
 - Kiểm tra và tiền xử lý output 
 - Tách dữ liệu thành 3 tập: train, validaton, test.
 - Tiến hành tiền xử lý


In [1]:
# IMPORT THƯ VIỆN
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

## Khám phá dữ liệu (đủ để có thể xác định câu hỏi)

In [2]:
pd.set_option('display.max_columns', None)  # Xem tất cả các cột

data = pd.read_csv('HOUSEs_info.csv', sep = '\t')
for i in range(0, data.shape[0]):
    print(data['Diện tích'][i])

83.15 m²
53.2 m²
105.8 m²
106.1 m²
60 m²
53.67 m²
69.42 m²
61 m²
53.5 m²
75.99 m²
71 m²
86 m²
96.4 m²
123.3 m²
65 m²
42 m²
59 m²
51 m²
74 m²
75.5 m²
81 m²
80 m²
57 m²
nan
47 m²
71 m²
65.6 m²
43.3 m²
69.16 m²
47.7 m²
78.24 m²
58 m²
80 m²
55 m²
90 m²
59 m²
62 m²
89.7 m²
75.6 m²
66 m²
73 m²
32.3
88 m²
70.68 m²
88 m²
60 m²
85 m²
59 m²
75 m²
69 m²
97 m²
75.69 m²
90 m²
72
54.72 m²
121.3 m²
39 m²
90 m²
76 m²
48.5 m²
97 m²
70
45 m²
65 m²
51.3 m²
30 m²
55.3 m²
53.2 m²
57.64 m²
56.5 m²
14.13 m²
102.1 m²
13.85 m²
33 m²
75 m²
300 m²
98.29 m²
48
56 m²
49.5 m²
57.64 m²
90.3 m²
92 m²
90 m²
50
127.6 m²
89 m²
29.5 m²
69.2 m²
85.52 m²
24.8 m²
82 m²
65 m²
79.2 m²
93.7 m²
69 m²
28 m²
82 m²
52 m²
53.8
40 m²
68 m²
68 m²
73 m²
26 m²
nan
78.88 m²
94.89 m²
50 m²
36.4 m²
68.6 m²
44
31 m²
48
81 m²
56.6 m²
30.3 m²
44.9 m²
76 m²
30 m²
59 m²
107 m²
85 m²
80.7 m²
57.64 m²
53.2 m²
59 m²
29 m²
117 m²
104.2 m²
102 m²
71 m²
69
72.41 m²
83.57 m²
66.66 m²
100 m²
71.42 m²
51.7 m²
1,202 m²
79.9 m²
66.52 m²
65.11 m²
50
49.73

67.2 m²
64 m²
59 m²
58
70 m²
451.3 m²
54 m²
50 m²
31.6 m²
31 m²
96
30.9 m²
71 m²
59 m²
57.1 m²
59.2 m²
74.9 m²
67.37 m²
56 m²
106.8 m²
20
96.6 m²
51.93 m²
82 m²
23.3 m²
83.42 m²
113 m²
41.5 m²
57.03 m²
80 m²
62 m²
40 m²
104 m²
60 m²
81 m²
53 m²
75 m²
69.1 m²
49 m²
71.5 m²
95.49 m²
34.5 m²
23.8 m²
52
53.2 m²
60
86 m²
8.3 m²
160 m²
71.36 m²
24.3 m²
50 m²
40
94.89 m²
80 m²
60.1 m²
30 m²
84 m²
109 m²
60 m²
30 m²
98.38 m²
97.2 m²
21
42
56 m²
74 m²
106
42 m²
87.5 m²
93.5 m²
59 m²
53.2 m²
59 m²
47 m²
57.2 m²
83.2 m²
54.2 m²
118.7 m²
49 m²
67 m²
67.1 m²
81.22 m²
106 m²
66 m²
120.8 m²
46 m²
72 m²
46.5 m²
93 m²
89.7 m²
72 m²
84 m²
43 m²
93 m²
100.35 m²
60
67 m²
81 m²
26.6 m²
53 m²
71.38 m²
20 m²
37.4
82 m²
67.37 m²
62.5 m²
58 m²
78 m²
56 m²
78.65 m²
68 m²
81 m²
56.58 m²
96 m²
73.49 m²
59 m²
81 m²
85 m²
84 m²
67 m²
84.95 m²
65.51 m²
75.99 m²
65 m²
36 m²
56.98 m²
71 m²
74 m²
66.6 m²
48 m²
48.4 m²
21 m²
54 m²
127 m²
50.57 m²
45.8 m²
60
50 m²
108 m²
43.5 m²
65 m²
66.66 m²
69 m²
70 m²
75 m²
54 m²
46 

75.83 m²
42 m²
37
46 m²
106 m²
85.03 m²
57
74 m²
69.5 m²
53.2 m²
52.13 m²
114 m²
53 m²
69 m²
88 m²
62 m²
69 m²
52 m²
52.02 m²
50 m²
57 m²
72 m²
60 m²
115 m²
72.23 m²
79.6 m²
81.78 m²
612 m²
51.15 m²
65.6 m²
89.12 m²
71.6 m²
66.66 m²
61 m²
45 m²
69 m²
82.82 m²
59 m²
354 m²
71.42 m²
69.18 m²
97.6 m²
52.54 m²
70.12 m²
98 m²
103.96 m²
46.7 m²
66.66 m²
39 m²
59 m²
120 m²
54.5 m²
60 m²
69 m²
47 m²
59.1 m²
90 m²
75 m²
66.66 m²
132.5 m²
62 m²
65 m²
88 m²
53.2 m²
52.6 m²
70 m²
104 m²
98.4 m²
79.9 m²
67 m²
47 m²
68 m²
44.2 m²
59 m²
76.5 m²
nan
64.5 m²
89.4 m²
30 m²
31 m²
91.6 m²
233.5 m²
63 m²
81 m²
82 m²
60.3 m²
124 m²
85.84 m²
51.6 m²
53.2 m²
92.5 m²
78.5 m²
80 m²
85.5 m²
74.77 m²
74 m²
75 m²
97.25 m²
54 m²
47 m²
66.66 m²
242 m²
60 m²
59 m²
97 m²
58.9 m²
69 m²
79.1 m²
59 m²
100.12 m²
60 m²
83 m²
65 m²
22 m²
82 m²
60 m²
148 m²
53.2 m²
78.57 m²
31.6 m²
nan
74.27 m²
42 m²
36.1 m²
79 m²
82 m²
57.28 m²
137.4 m²
49 m²
153.87 m²
86.54 m²
97 m²
75.66 m²
52.13 m²
133 m²
56 m²
88 m²
59 m²
137.43 m²
65.5

57.2 m²
nan
94.89 m²
45.5 m²
43.4 m²
76.2 m²
58.8 m²
76 m²
285 m²
41.2 m²
93 m²
107 m²
83 m²
59 m²
66.66 m²
30 m²
75 m²
104.03 m²
37.5 m²
54 m²
68 m²
110.3 m²
57 m²
67 m²
78 m²
70 m²
33.6 m²
107 m²
51 m²
73 m²
49 m²
76 m²
38 m²
46.7 m²
nan
69 m²
101.5 m²
72 m²
63 m²
80.14 m²
55.86 m²
20.9 m²
100 m²
71.96 m²
69 m²
226.2 m²
104.03 m²
57 m²
66.69 m²
65 m²
60 m²
68 m²
63 m²
56 m²
131.3 m²
74 m²
47 m²
52 m²
66 m²
95.68 m²
80 m²
55
87.12 m²
101 m²
69.1 m²
68 m²
83.6 m²
475.9 m²
69 m²
118 m²
120 m²
65 m²
50 m²
37.1 m²
47 m²
35.7 m²
34.85 m²
52 m²
50 m²
75 m²
79 m²
73.75 m²
45.97 m²
19.7 m²
105 m²
174 m²
65 m²
92 m²
100.35 m²
46.8 m²
69.2 m²
154 m²
67.4 m²
70 m²
22
64 m²
64 m²
77 m²
59 m²
46.5 m²
134.7 m²
73 m²
72 m²
48
66 m²
122.14 m²
82 m²
59 m²
88 m²
66.2 m²
92 m²
55
35.4
49.3 m²
80.4 m²
73 m²
75 m²
80.6 m²
74.77 m²
65 m²
83 m²
33 m²
49.1 m²
141
89.9 m²
56 m²
77.2 m²
67 m²
69.9 m²
89 m²
75
61 m²
48 m²
58 m²
60 m²
28.2 m²
66.66 m²
66.84 m²
93.5 m²
71 m²
100.3 m²
77.8 m²
51
47 m²
106 m²
110 m

### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [3]:
data.shape


(7154, 120)

### Dữ liệu có các dòng bị lặp không?

In [4]:
data.index.duplicated().sum()

0

## Đưa ra câu hỏi cần trả lời
*Output - Giá bán -* được tính từ *input - các thông tin của hành khách -* theo công thức nào?
 Trong đồ án này, ý nghĩa thực tế của việc đi tìm câu trả lời cho câu hỏi trên giúp người dùng có được 1 sự tham khảo với 1 độ chính xác cụ thể về giá nhà dựa trên các thông tin. Qua đó, người dùng có thể dựa trên dự đoán để tự đưa ra quyết định về việc mua (bán) nhà với một mức giá hợp lý.  


## Khám phá dữ liệu 
### 1. OUTPUT TRƯỚC KHI TIỀN XỬ LÝ

In [5]:
# Cột output hiện có kiểu dữ liệu gì?
data['Giá bán'].dtype

dtype('O')

In [6]:
# Cột output có giá trị thiếu không?
data['Giá bán'].isna().sum()

396

#### TIỀN XỬ LÝ OUTPUT

In [7]:
# Xoá ouput có giá trị thiếu
data = data[data['Giá bán'].notna()]
data['Giá bán'].isna().sum()
data = data.reset_index(drop = True) 

In [8]:
# Chuẩn hoá về dạng số

# Vị trí các số hảng tỷ
mil_index = [] 
# Vị trí các số hảng triệu
bil_index = []

for i in range(0, data.shape[0]):
    value = data['Giá bán'][i] 
    if 'tỷ' in value:
        data.at[i, 'Giá bán'] = value.rstrip('tỷ')
        bil_index.append(i)
    elif 'tr' in value:
        data.at[i, 'Giá bán'] = value.rstrip(' tr')
        mil_index.append(i)

In [9]:
# Chuyển đơn vị triệu thành tỷ (đồng)
data['Giá bán'] = data['Giá bán'].astype(float)
for i in mil_index:
    data.at[i, 'Giá bán'] = data['Giá bán'][i] / 1000

### 2. SAU KHI TIỀN XỬ LÝ OUTPUT

In [10]:
# Cột output hiện có kiểu dữ liệu gì?
data['Giá bán'].dtype

dtype('float64')

In [11]:
# Cột output có giá trị thiếu không?
data['Giá bán'].isna().sum()

0

## Tiền xử lý (tách các tập)
    Ta tách các tập từ dữ liệu thu thập được thành 3 tập: train, validation và test theo tỉ lệ 70%:15%:15%

In [12]:
# Tách X và y
y_sr = data['Giá bán'] # sr là viết tắt của series
X_df = data.drop("Giá bán", axis=1)

In [13]:
# Tách theo tỉ lệ: 7:1.5:1.5
# Tách dữ liệu thu thập thành 2 tập: tập huấn luyện và tập other (validation và test) theo tỉ lệ 70%:30%
train_X_df, other_X_df, train_y_sr, other_y_sr = train_test_split(X_df, y_sr, test_size=0.3, random_state=0)

# Tách tập other (validation và test) thành 2 tập: tập validation và tập test theo tỉ lệ 50%:50% (tức: bằng 15%:15% so với dữ liệu ban đầu)
val_X_df, test_X_df, val_y_sr, test_y_sr = train_test_split(other_X_df, other_y_sr, test_size=0.5, random_state=0)

train_X_df.shape, val_X_df.shape, test_X_df.shape, train_y_sr.shape, val_y_sr.shape, test_y_sr.shape

((4730, 119), (1014, 119), (1014, 119), (4730,), (1014,), (1014,))

## Khám phá dữ liệu (tập huấn luyện)

In [14]:
train_X_df.dtypes

Loại hình                object
Phòng ngủ               float64
Phòng tắm               float64
Diện tích                object
Tình hình nội thất       object
                         ...   
Quạt trần hàng Ngoại     object
Hồ cá                    object
3 máy lạnh               object
1 phòng thờ              object
1 sân phơi               object
Length: 119, dtype: object

In [15]:
train_X_df.dtypes[train_X_df.dtypes != object]

Phòng ngủ    float64
Phòng tắm    float64
Số tầng      float64
dtype: object

In [16]:
dropped_cols = []
num_cols = ['Phòng ngủ', 'Phòng tắm', 'Số tầng']

In [17]:
# Các cột dạng số
df = train_X_df[num_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def lower_quartile(df):
    return df.quantile(0.25).round(1)
def median(df):
    return df.quantile(0.5).round(1)
def upper_quartile(df):
    return df.quantile(0.75).round(1)
df.agg([missing_ratio, 'min', lower_quartile, median, upper_quartile, 'max'])

,Phòng ngủ,Phòng tắm,Số tầng
missing_ratio,2.0,2.4,47.6
min,0.0,1.0,1.0
lower_quartile,2.0,1.0,3.0
median,2.0,2.0,9.0
upper_quartile,2.0,2.0,17.0
max,31.0,32.0,44.0


In [18]:
# Các cột không phải dạng số

cat_cols = list(set(train_X_df.columns) - set(num_cols))
df = train_X_df[cat_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def num_values(df):
    return df.nunique()
def value_ratios(c):
    return dict((c.value_counts(normalize=True) * 100).round(1))
df.agg([missing_ratio, num_values, value_ratios])


,Bàn trang điểm,Bàn ghế phòng khách,Loại chủ quyền,Căn hộ hạng sang,Máy sấy,Khóa cửa mật khẩu,Camera an ninh,Khóa cửa thẻ từ,Nhà trống,Căn hộ cao cấp quận 2,Bồn tắm,Giường\n,Giếng trời,Diện tích sử dụng,Tủ giày dép,Sàn gỗ,Rèm,tường,Phòng giải trí,Máy lạnh,Nuôi thú cưng,Mục đích sử dụng,Sân vườn,Tháp,Điện,Tủ treo đồ / tủ quần áo,Đèn trần,Trạng thái,Diện tích sàn,Thời gian bắt đầu bán,Căn hộ Thảo Điền,Giường,Hồ bơi tràn,"Bếp ga, bếp từ",căn hộ,Máy nước nóng,Số phòng ngủ,Tủ âm tường,Số phòng tắm,3 máy lạnh,Đảo bếp\n,Diện tích,Lò nướng,Kệ bếp,Gác Lửng,Kệ sách,Hoàn thiện cơ bản,Tủ lạnh,Dàn phơi đồ thông minh,Chiều rộng,Khóa cửa vân tay,Tủ bếp trên\n,sàn,tủ âm,Bộ bàn ăn và ghế,Tủ bếp dưới,Thiết bị âm thanh,Loại hình nhà đất,Hẻm Xe Hơi,Căn hộ view sông,căn hộ quận 7,Hướng cửa,khuôn viên đi dạo,Chủ đầu tư Singapore Thảo Điền,Ban công,Tình trạng nội thất,Đèn trang trí,Phòng cho giúp việc,Era Town quận 7,Mã nhà đất,Đèn,1 sân phơi,Gym,Quầy minibar,Bếp ướt,Máy giặt,Tủ bếp dưới\n,Thời gian Xây dựng,Bàn làm việc,Mặt tiền chợ,Tủ kê đầu giường,Đảo bếp,Giá đăng,Kệ TV,Bếp khô,Căn Hộ Era Town,Ngày Đăng,Đèn bàn,Đầy đủ vệ sinh,Góc làm việc,Chỗ đậu xe hơi,Nhà kho,Loại hình,"Đệm, chăn, gối",Tủ bếp trên,Tầng hầm,Máy hút mùi,Smart Home,Truyền hình cáp,Quạt trần hàng Ngoại,Khu tiệc nướng BBQ,căn hộ cần bán,Internet,Vườn tiểu cảnh,1 phòng thờ,Khu vui chơi trẻ em,Hồ cá,Hướng ban công,Cảm biến cháy,Tình hình nội thất,TV,Q2 Thảo Điền,Hồ bơi riêng,Dự án,Lò vi sóng,Chiều dài
missing_ratio,2,2,28.5,100,0,0,0,0,100,100,2,99.5,100,18.3,2,2,100,100,0,0,0,94.3,0,53.3,100,2,2,78.7,99.3,85.5,100,2.4,100,0,100,0,100,2,100,100,99.6,0.7,0,100,100,2,100,0,0,98.8,0,99.7,100,100,2,2.3,0,100,100,100,100,100,100,100,0,100,2,0,100,100,100,100,0,0,2,0,99.7,95.4,2,100,2,2.3,100,2,2,100,100,2,100,0,0,0,0,2,2.2,0,0,0,0,100,100,100,0,100,100,100,100,100,0,4.7,0,100,0,18.2,0,98.8
num_values,2,2,4,1,2,2,2,2,1,1,2,2,0,1019,2,2,1,0,2,2,2,23,2,227,0,2,2,1,32,396,1,2,0,2,1,2,0,2,0,0,1,1140,2,1,0,2,1,2,2,17,2,1,0,0,2,2,2,0,1,1,1,0,0,1,2,0,2,2,1,0,0,0,2,2,2,2,1,31,2,1,2,2,0,2,2,1,0,2,1,2,2,2,14,2,2,2,2,2,2,0,0,1,2,0,0,0,0,0,2,3,2,1,2,351,2,23
value_ratios,"{'No': 95.9, 'Yes': 4.1}","{'No': 91.9, 'Yes': 8.1}","{'HĐ mua bán': 72.6, 'Sổ hồng': 25.4, 'Sổ đỏ':...",{'Yes': 100.0},"{'No': 99.7, 'Yes': 0.3}","{'No': 99.3, 'Yes': 0.7}","{'No': 99.0, 'Yes': 1.0}","{'No': 99.1, 'Yes': 0.9}",{'Yes': 100.0},{'Yes': 100.0},"{'No': 99.4, 'Yes': 0.6}","{'No': 95.5, 'Yes': 4.5}",{},"{'54 m²': 3.1, '63 m²': 3.0, '65 m²': 2.0, '75...","{'No': 99.0, 'Yes': 1.0}","{'No': 99.2, 'Yes': 0.8}",{'Yes': 100.0},{},"{'No': 97.0, 'Yes': 3.0}","{'No': 97.9, 'Yes': 2.1}","{'No': 96.5, 'Yes': 3.5}","{'để ở hoặc cho thuê': 30.1, 'Ở': 23.0, 'Để ở'...","{'No': 97.6, 'Yes': 2.4}","{'Block B': 3.4, 'Tháp A': 3.1, 'A': 2.9, 'B':...",{},"{'No': 91.1, 'Yes': 8.9}","{'No': 98.6, 'Yes': 1.4}",{'Chưa bàn giao': 100.0},"{'160 m²': 8.8, '356 m²': 2.9, '400 m²': 2.9, ...","{'28/09/2020': 1.9, '07/12/2020': 1.5, '16/12/...",{'Yes': 100.0},"{'No': 92.3, 'Yes': 7.7}",{},"{'No': 98.8, 'Yes': 1.2}",{'Yes': 100.0},"{'No': 98.9, 'Yes': 1.1}",{},"{'No': 98.9, 'Yes': 1.1}",{},{},{'No': 100.0},"{'59 m²': 3.1, '69 m²': 2.3, '70 m²': 1.9, '75...","{'No': 99.6, 'Yes': 0.4}",{'Yes': 100.0},{},"{'No': 98.3, 'Yes': 1.7}",{'Yes': 100.0},"{'No': 98.3, 'Yes': 1.7}","{'No': 99.5, 'Yes': 0.5}","{'4 m': 26.8, '5 m': 14.3, '8 m': 12.5, '10 m'...","{'No': 99.4, 'Yes': 0.6}",{'No': 100.0},{},{},"{'No': 92.3, 'Yes': 7.7}","{'No': 98.0, 'Yes': 2.0}","{'No': 99.8, 'Yes': 0.2}",{},{'Yes': 100.0},{'Yes': 100.0},{'Yes': 100.0},{},{},{'Yes': 100.0},"{'No': 87.7, 'Yes': 12.3}",{},"{'No': 98.6, 'Yes': 1.4}","{'No': 99.3, 'Yes': 0.7}",{'Yes': 100.0},{},{},{},"{'No': 90.2, 'Yes': 9.8}","{'No': 99.3, 'Yes': 0.7}","{'No': 99.4, 'Yes': 0.6}","{'No': 98.7, 'Yes': 1.3}",{'No': 100.0},"{'01/01/2016': 19.6, '01/01/2015': 16.4, '01/0...","{'No': 95.2, 'Yes': 4.8}",{'Yes': 100.0},"{'No': 99.3, 'Yes': 0.7}","{'No': 99.8, 'Yes': 0.2}",{},"{'No': 98.1, 'Yes': 1.9}","{'No': 98.6, 

In [19]:
# liệt kê các tập có độ missing values >= 50%

for i in range(0, df.shape[1]):
    if df.agg(missing_ratio)[i] >= 50:
#         print(df.columns.values[i], df.agg(missing_ratio)[i])
        dropped_cols.append(df.columns.values[i])
len(dropped_cols)

56

## Tiền xử lý (tập huấn luyện)

    Ta tiến hành tiền xử lý như sau:
       * Xoá đi những cột có độ missing values > 50%
       * Chuyển các cột dạng số nhưng lại ở dạng chuỗi về lại dạng số ('Diệc tích', 'Diện tích sử dụng')
       

In [20]:
# Hàm chuyển các cột dạng số nhưng lại ở dạng chuỗi về lại dạng số
def ConVert2Num(df, array):
    '''
    Input : df, Categorical Column Name Array
    Output: Numeric Columns
    '''
    
    for j in array:
        for i in range(0, df.shape[0]):
            index = df.index[i]
            temp = df[j][index] 

            if pd.isna(temp) == False:
                if 'm²' in temp:
                    df.at[index, j] = temp.replace(',', '').rstrip(' m²')
                    
        df[j] = df[j].astype(float)
    
    return df

# Hàm chuyển cột số tầng từ float sang int
# def Convert2Int(df, array):
#     '''
#     Input : df, Float Column Name Array
#     Output: Int Columns
#     '''
    
#     for i in array:
#         for j in range(0, df.shape[0]):
#             index = df.index[j] 

#             if pd.isna(df[i][index]) == False:
#                     df.at[index, i] = int(df[i][index])
# #     df[i] = df[i].astype(int)

#     print(df['Số tầng'].dtype)
#     return df

In [21]:
class ColAdderDropper(BaseEstimator, TransformerMixin):
    def __init__(self, num_top_Loai_hinh=1, num_top_Du_an=1):
        self.num_top_Loai_hinh = num_top_Loai_hinh
        self.num_top_Du_an = num_top_Du_an

    def fit(self, X_df, y=None):
        Loai_hinh_col = X_df['Loại hình']
        self.Loai_hinh_counts_ = Loai_hinh_col.value_counts()
        Loai_hinh = list(self.Loai_hinh_counts_.index)
        self.top_Loai_hinh_ = Loai_hinh[:max(1, min(self.num_top_Loai_hinh, len(Loai_hinh)))]

        Du_an_col = X_df['Dự án']
        self.Du_an_counts_ = Du_an_col.value_counts()
        Du_an = list(self.Du_an_counts_.index)
        self.top_Du_an_ = Du_an[:max(1, min(self.num_top_Du_an, len(Du_an)))]

        return self
    def transform(self, X_df, y=None):
        # YOUR CODE HERE
        df = X_df.copy()
        Loai_hinh_col = df["Loại hình"] 
        Loai_hinh_col[~Loai_hinh_col.isin(self.top_Loai_hinh_)] = 'Others'
        df["Thuộc loại hình"] = Loai_hinh_col
        
        Du_an_col =  df["Dự án"] 
        Du_an_col[~Du_an_col.isin(self.top_Du_an_)] = 'Others'
        df["Thuộc dự án"] = Du_an_col
        
        dropped_cols.extend(('Loại hình', 'Dự án'))
        df.drop(dropped_cols, axis=1, inplace=True)
        
        to_float_arr = ['Diện tích', 'Diện tích sử dụng']
        ConVert2Num(df, to_float_arr)
        
#         to_int_arr = ['Số tầng']
#         Convert2Int(df, to_int_arr)
        return df

In [22]:
# TEST FIT METHOD
col_adderdropper = ColAdderDropper(num_top_Loai_hinh=4, num_top_Du_an=4)
col_adderdropper.fit(train_X_df)
print(col_adderdropper.Loai_hinh_counts_)
print()
print(col_adderdropper.Du_an_counts_)
print()
print(col_adderdropper.top_Loai_hinh_)
print()
print(col_adderdropper.top_Du_an_)

Căn hộ                 3811
Nhà phố                 583
Office-tel              173
Đất nền                  39
Shophouse                38
Biệt thự                 27
Penthouse                23
Văn phòng                20
Căn hộ dịch vụ            6
Loại khác                 4
Lofthouse                 3
Đất công nghiệp           1
Mặt bằng kinh doanh       1
Nhà xưởng kho bãi         1
Name: Loại hình, dtype: int64

Vinhomes Grand Park       623
Vinhomes Central Park     238
Q7 Saigon Riverside       201
The Sun Avenue            185
Saigon South Residence    151
                         ... 
Melody Residences           1
Osimi Tower                 1
The Golden Dynasty          1
Phố Đông Hoa Sen            1
Chung cư H1 Hoàng Diệu      1
Name: Dự án, Length: 351, dtype: int64

['Căn hộ', 'Nhà phố', 'Office-tel', 'Đất nền']

['Vinhomes Grand Park', 'Vinhomes Central Park', 'Q7 Saigon Riverside', 'The Sun Avenue']


In [23]:
after_train_transform_df = col_adderdropper.transform(train_X_df)

<ipython-input-21-94d2372d8fb3>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Loai_hinh_col[~Loai_hinh_col.isin(self.top_Loai_hinh_)] = 'Others'
<ipython-input-21-94d2372d8fb3>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Du_an_col[~Du_an_col.isin(self.top_Du_an_)] = 'Others'


In [24]:
num_cols.extend(('Diện tích', 'Diện tích sử dụng'))
num_cols

['Phòng ngủ', 'Phòng tắm', 'Số tầng', 'Diện tích', 'Diện tích sử dụng']

In [25]:
cate_cols = []
for i in after_train_transform_df.columns:
    if i not in num_cols:
        cate_cols.append(i)
        print(i)
len(cate_cols)

Tình hình nội thất
Ban công
Phòng cho giúp việc
Sân vườn
Phòng giải trí
Chỗ đậu xe hơi
Hồ bơi riêng
Quầy minibar
Tầng hầm
Góc làm việc
Nhà kho
Nuôi thú cưng
Gym
Bàn ghế phòng khách
Kệ TV
Bộ bàn ăn và ghế
Đảo bếp
Đèn trang trí
Bếp ướt
Bếp khô
Tủ bếp trên
Tủ bếp dưới
Sàn gỗ
Tủ âm tường
Tủ treo đồ / tủ quần áo
Tủ kê đầu giường
Đèn trần
Đèn bàn
Giường
Đệm, chăn, gối
Tủ giày dép
Bồn tắm
Kệ sách
Bàn làm việc
Bàn trang điểm
TV
Máy hút mùi
Cảm biến cháy
Tủ lạnh
Bếp ga, bếp từ
Máy giặt
Máy sấy
Dàn phơi đồ thông minh
Máy lạnh
Máy nước nóng
Khóa cửa thẻ từ
Khóa cửa mật khẩu
Khóa cửa vân tay
Thiết bị âm thanh
Smart Home
Lò nướng
Lò vi sóng
Camera an ninh
Internet
Truyền hình cáp
Loại chủ quyền
Thuộc loại hình
Thuộc dự án


58

In [26]:
imp_mean = SimpleImputer(strategy='mean')
imp_mode = SimpleImputer(strategy='most_frequent')
encoding = OneHotEncoder()
categorical_transformer = make_pipeline(imp_mode, encoding)

colTransform = ColumnTransformer(transformers=[('numerical', imp_mean, num_cols),
                                               ('categorical', categorical_transformer, cate_cols)])

colNormalize = StandardScaler()
preprocess_pipeline = make_pipeline(col_adderdropper, colTransform, colNormalize)

preprocessed_train_X = preprocess_pipeline.fit_transform(train_X_df)

<ipython-input-21-94d2372d8fb3>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Loai_hinh_col[~Loai_hinh_col.isin(self.top_Loai_hinh_)] = 'Others'
<ipython-input-21-94d2372d8fb3>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Du_an_col[~Du_an_col.isin(self.top_Du_an_)] = 'Others'


In [27]:
preprocess_pipeline

Pipeline(steps=[('coladderdropper',
                 ColAdderDropper(num_top_Du_an=4, num_top_Loai_hinh=4)),
                ('columntransformer',
                 ColumnTransformer(transformers=[('numerical', SimpleImputer(),
                                                  ['Phòng ngủ', 'Phòng tắm',
                                                   'Số tầng', 'Diện tích',
                                                   'Diện tích sử dụng']),
                                                 ('categorical',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncod...
                                                   'Sân vườn', 'Phòng giải trí',
                                                   'Chỗ đậu xe hơi',
                                                   'Hồ bơi riêng',
                                                   'Quầy minibar', 'Tầng hầm',
                                                   'Góc làm việc', 'Nhà kho',
                                                   'Nuôi thú cưng', 'Gym',
                                                   'Bàn ghế phòng khách',
                                                   'Kệ TV', 'Bộ bàn ăn và ghế',
                                                   'Đảo bếp', 'Đèn trang trí',
                                                   'Bếp ướt', 'Bếp khô',
                                                   'Tủ bếp trên', 'Tủ bếp dưới',
                                                   'Sàn gỗ', 'Tủ âm tường',
                                                   'Tủ treo đồ / tủ quần áo',
                                                   'Tủ kê đầu giường',
                                                   'Đèn trần', 'Đèn bàn',
                                                   'Giường', 'Đệm, chăn, gối', ...])])),
                ('standardscaler', StandardScaler())])

In [28]:
preprocessed_val_X = preprocess_pipeline.transform(val_X_df)

<ipython-input-21-94d2372d8fb3>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Loai_hinh_col[~Loai_hinh_col.isin(self.top_Loai_hinh_)] = 'Others'
<ipython-input-21-94d2372d8fb3>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Du_an_col[~Du_an_col.isin(self.top_Du_an_)] = 'Others'


In [29]:
preprocessed_val_X = preprocess_pipeline.transform(test_X_df)

<ipython-input-21-94d2372d8fb3>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Loai_hinh_col[~Loai_hinh_col.isin(self.top_Loai_hinh_)] = 'Others'
<ipython-input-21-94d2372d8fb3>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Du_an_col[~Du_an_col.isin(self.top_Du_an_)] = 'Others'
